# Introduction to the Planner

The Planner is one of the fundamental concepts of the Semantic Kernel. It makes use of the collection of plugins that have been registered to the kernel and using AI, will formulate a plan to execute a given ask.

Read more about it [here](https://aka.ms/sk/concepts/planner).

In [2]:
#r "nuget: Microsoft.SemanticKernel, 1.11.1"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.11.1-preview"

#!import config/Settings.cs
#!import config/Utils.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

var builder = Kernel.CreateBuilder();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
else
    builder.AddOpenAIChatCompletion(model, apiKey, orgId);

var kernel = builder.Build();

Installed Packages Microsoft.SemanticKernel, 1.11.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.11.1-preview

### Setting Up Handlebars Planner
Handlebars Planner is located in the `Microsoft.SemanticKernel.Planning.Handlebars` package.

In [3]:
using Microsoft.SemanticKernel.Planning.Handlebars;

#pragma warning disable SKEXP0060

var planner = new HandlebarsPlanner();

### Providing plugins to the planner
The planner needs to know what plugins are available to it. Here we'll import the `SummarizePlugin` and `WriterPlugin` we have defined on disk.

In [4]:
var pluginsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "..", "plugins");

var kernel = builder.Build();
kernel.ImportPluginFromPromptDirectory(Path.Combine(pluginsDirectory, "GMToolsPlugin"));


Define your ASK. What do you want the Kernel to do?

In [5]:
#pragma warning disable SKEXP0060

var ask = @"
You are a game master for a Dungeons and Dragons campaign.
Please create a five-room dungeon for a party of 4 level 5 characters.
The five-room dungeon should be of medium difficulty and take place in a forest.
The party consists of a fighter, a wizard, a cleric, and a rogue.
The party is on a quest to retrieve a magical artifact from a bandit camp.
Ensure that the output includes:
* the five-room dungeon outline with appropriate markdown headers, and
* descriptive text for each room with appropriate markdown headers.
";
var originalPlan = await planner.CreatePlanAsync(kernel, ask);

Console.WriteLine("Original plan:\n");
Console.WriteLine(originalPlan);

Original plan:

{{!-- Step 0: Extract key values --}}
{{set "gameSystem" "Dungeons and Dragons"}}
{{set "partyContext" "A party of 4 level 5 characters consisting of a fighter, a wizard, a cleric, and a rogue, on a quest to retrieve a magical artifact from a bandit camp in a forest. The dungeon should be of medium difficulty."}}
{{set "roomNumbers" (array "1" "2" "3" "4" "5")}}

{{!-- Step 1: Generate the Five Room Dungeon outline --}}
{{set "dungeonOutline" (GMToolsPlugin-FiveRoomDungeon game_system=gameSystem input_context=partyContext)}}

{{!-- Step 2: Generate descriptive texts for each room --}}
{{#each roomNumbers}}
### Room #{{this}}
{{set "roomDescription" (GMToolsPlugin-RoomDescription input_context=dungeonOutline room_number=this)}}
{{json roomDescription}}
{{/each}}


As you can see in the above plan, the Planner has taken the user's ask and converted it into a Plan object detailing how the AI would go about solving this task.

It makes use of the plugins that the Kernel has available to it and determines which functions to call in order to fulfill the user's ask.

Let's also define an inline plugin and have it be available to the Planner.
Be sure to give it a function name and plugin name.

### Executing the plans

Now that we have different plans, let's try to execute them! The Kernel can execute the plan using RunAsync.

In [26]:
#pragma warning disable SKEXP0060

var originalPlanResult = await originalPlan.InvokeAsync(kernel, new KernelArguments());

Console.WriteLine("Original Plan results:\n");
Console.WriteLine(originalPlanResult.ToString());

Original Plan results:

Five Room Dungeon Outline: ## Room 1: Entrance and Guardian
* **Objective:** Create an initial challenge.
* **Design:** The party arrives at the edge of the bandit camp, which is hidden deep within the forest. The entrance is guarded by two bandit sentries and a trained wolf. The sentries are alert and have set up a rudimentary alarm system using tripwires and bells.

**Description:** The dense forest opens up to a small clearing where the bandit camp is located. The camp is surrounded by a makeshift wooden palisade, with a single entrance guarded by two bandits and a wolf. The air is tense, and the sounds of the forest are muted by the presence of danger.

## Room 2: Puzzle or Roleplaying Challenge
* **Objective:** Engage players with non-combat interaction.
* **Design:** Inside the camp, the party encounters a bandit who is willing to negotiate. This bandit, disillusioned with the bandit leader, offers to help the party in exchange for safe passage out of the 

Now lets execute and print the new plan: